### 发现就是CVRP问题：<p>（1）参数：导入excel文件数据<p>（2）加权因子function编写：考虑_距离/节点规模_步兵坦克数量/障碍规避_威胁估计<p>（3）“ant model”：工人数量/速度_车辆<p>(4)物资承载量_是否分类？医疗日用品/军事，建议要分也之分两类：坦克_军事补充_以坦克数量进行量化模拟；步兵_医疗等日用品_以步兵数量进行量化模拟<p>（5）物资节点的设定：后方“空降”<p>（6）


### (MMAS初步)[https://blog.csdn.net/qq_36744449/article/details/120033201]

In [1]:
import numpy as np
from numpy import random as rd
import matplotlib.pyplot as plt
import pandas as pd
import time
np.set_printoptions(linewidth=1000, suppress=True)

# 从TSPLIB官网上下载TSP数据kroA100
with open("./kroA100.tsp", "r", encoding="utf-8") as file:
    data = file.read()
data = np.array([i.split(" ")[1:] for i in data.split("\n")[6:-2]]).astype(np.float)


# In[90]:

def GetTime(func_name):
    def inner(*args, **kwargs):
        start_time = time.time()
        ret = func_name(*args, **kwargs)
        end_time = time.time()
        run_time = end_time - start_time
        print("运行时间%f秒" % run_time)
        return ret
    return inner


# In[91]:

class MMAS(object):
    
    def __init__(self, city_position, initial_pheromone, max_pheromone, min_pheromone, run_times, ant_count, alpha, beta, Q, w, tau, q_0):
        """
            city_position:城市座标，N行2列，N表示城市个数，2列分别为x座标以及y座标
            initial_pheromone:初始信息素值
            max_pheromone:信息素最大值
            min_pheromone:信息素最小值
            run_times:算法跑几代
            ant_count:蚂蚁数量
            alpha:选择城市时信息素重要性
            beta:选择城市时启发信息重要性
            Q:信息素更新时用到的常数
            w:信息素的挥发百分比, 0到1之间
            tau:信息素平滑系数，0到1之间
            q_0:每只蚂蚁在状态转移时采取随机型还是确定型，0到1之间。随机产生一个0到1之间的随机数，
                如果随机数大于q_0，则可能选择被选概率最大的城市；如果小于q_0，则一定选择被选概率最大的城市
        """
        self.city_position = city_position
        self.max_pheromone = max_pheromone
        self.min_pheromone = min_pheromone
        self.run_times = run_times
        self.ant_count = ant_count
        self.city_count = self.city_position.shape[0]
        self.distance_matrix, self.heuristic_information = self.get_distance_matrix()
        self.pheromone_matrix = np.ones_like(self.distance_matrix) * initial_pheromone - np.diag([initial_pheromone] * self.city_count, k=0)
        self.alpha = alpha
        self.beta = beta
        self.current_best_path = None
        self.current_best_distance = float("inf")
        self.Q = Q
        self.w = w
        self.tau = tau
        self.best_distance_lst = []
        self.q_0 = q_0
    
    def get_distance_matrix(self):
        """
            根据城市座标获取城市距离矩阵，启发信息矩阵
        """
        for i in range(self.city_position.shape[0]):
            distance = np.sqrt(np.sum(np.power(self.city_position[i] - self.city_position, 2), axis=1, keepdims=True))
            if not i:
                result = distance
                continue
            result = np.concatenate([result, distance], axis=1)
        return result, pd.DataFrame(1 / result).replace(float("inf"), 0).values
    
    def get_proba(self, not_passed_city, passed_city):
        """
            获取所有未经过城市被选取的概率
        """
        not_passed_city = np.array(not_passed_city)
        current_city_of_every_ant = np.array(passed_city)[:, -1]
        pheromone_next_path = []
        heuristic_next_path = []
        for i in range(self.ant_count):
            pheromone_next_path.append(self.pheromone_matrix[[current_city_of_every_ant[i]] * not_passed_city.shape[1], not_passed_city[i]])
            heuristic_next_path.append(self.heuristic_information[[current_city_of_every_ant[i]] * not_passed_city.shape[1], not_passed_city[i]])
        pheromone_heuristic = np.power(pheromone_next_path, self.alpha) * np.power(heuristic_next_path, self.beta)
        prob = pheromone_heuristic / np.sum(pheromone_heuristic, axis=1, keepdims=True)
        return prob
    
    def get_path_distance(self, passed_city):
        """
            遍历完所有城市后，获取每只蚂蚁行走路径长度
        """
        distance_of_every_ant = [0] * self.ant_count
        for i in range(self.ant_count):
            for j in range(self.city_count - 1):
                start_city = passed_city[i][j]
                end_city = passed_city[i][j + 1]
                distance_of_every_ant[i] += self.distance_matrix[start_city, end_city]
            distance_of_every_ant[i] += self.distance_matrix[passed_city[i][-1], passed_city[i][0]]
        return np.array(distance_of_every_ant)
    
    def get_pheromone_update_matrix(self, current_generation_best_path, current_generation_best_distance):
        """
            获取信息素更新矩阵
        """
        pheromone_update_matrix = np.zeros_like(self.pheromone_matrix)
        if self.current_best_path == current_generation_best_path:
            # 当前代最优路径和当前最优路径为同一条路径
            current_update_value = self.Q / self.current_best_distance
            for i in range(self.city_count - 1):
                start = self.current_best_path[i]
                end = self.current_best_path[i + 1]
                pheromone_update_matrix[start, end] += current_update_value
            pheromone_update_matrix[self.current_best_path[-1], self.current_best_path[0]] += current_update_value
        else:
            # 当前代最优路径和当前最优路径不是同一条路径
            current_generation_update_value = self.Q / current_generation_best_distance
            current_update_value = self.Q / self.current_best_distance
            for i in range(self.city_count - 1):
                current_start = self.current_best_path[i]
                current_generation_start = current_generation_best_path[i]
                current_end = self.current_best_path[i + 1]
                current_generation_end = current_generation_best_path[i + 1]
                pheromone_update_matrix[current_start, current_end] += current_update_value
                pheromone_update_matrix[current_generation_start, current_generation_end] += current_generation_update_value
            pheromone_update_matrix[self.current_best_path[-1], self.current_best_path[0]] += current_update_value
            pheromone_update_matrix[current_generation_best_path[-1], current_generation_best_path[0]] += current_generation_update_value
        return pheromone_update_matrix
    
    def update_pheromone_matrix(self, pheromone_update_matrix):
        """
            信息素矩阵更新
        """
        self.pheromone_matrix = (1 - self.w) * self.pheromone_matrix + pheromone_update_matrix
        bigger_than_max_index = self.pheromone_matrix > self.max_pheromone
        smaller_than_min_index = self.pheromone_matrix < self.min_pheromone
        self.pheromone_matrix[bigger_than_max_index] = self.max_pheromone
        self.pheromone_matrix[smaller_than_min_index] = self.min_pheromone
        # 信息素平滑
        self.pheromone_matrix += (self.tau * (self.max_pheromone - self.pheromone_matrix))
        self.pheromone_matrix = self.pheromone_matrix - np.diag([self.pheromone_matrix[0, 0]] * self.pheromone_matrix.shape[0])
    
    @GetTime
    def run(self):
        """
            返回最佳路径以及路径长度
        """
        for i in range(1, self.run_times + 1):
            # 构造蚂蚁已经过城市列表passed_city，以及未经过城市列表not_passed_city
            passed_city = [[rd.randint(0, self.city_count)] for i in range(self.ant_count)]
            not_passed_city = [list(set(range(self.city_count)) - set(i)) for i in passed_city]
            # 当存在未遍历的城市就执行循环体,直到所有的城市遍历完跳出循环
            while np.unique(not_passed_city).shape[0]:
                # 选择下一个城市
                select_prob = self.get_proba(not_passed_city, passed_city)
                q = rd.random()
                if q > self.q_0:
                    # 随机型
                    cum_select_prob = np.cumsum(select_prob, axis=1)
                    select_city_index = []
                    for i in range(self.ant_count):
                        rand_num = rd.random()
                        select_city_index.append(list(rand_num < cum_select_prob[i]).index(True))
                else:
                    # 确定型
                    select_city_index = np.argmax(select_prob, axis=1)
                for i in range(self.ant_count):
                    passed_city[i].append(not_passed_city[i].pop(select_city_index[i]))
            # 混合方式更新信息素
            distance_of_every_ant = self.get_path_distance(passed_city)
            # 找出当前代最短路径及其长度
            best_index = np.argmin(distance_of_every_ant)
            current_generation_best_path = passed_city[best_index]
            current_generation_best_distance = distance_of_every_ant[best_index]
            # 更新当前最优路径及其长度
            if current_generation_best_distance < self.current_best_distance:
                self.current_best_distance = current_generation_best_distance
                self.current_best_path = current_generation_best_path
            pheromone_update_matrix = self.get_pheromone_update_matrix(current_generation_best_path, current_generation_best_distance)
            # 更新信息素矩阵
            self.update_pheromone_matrix(pheromone_update_matrix)
            self.best_distance_lst.append(self.current_best_distance)
        return self.current_best_path, self.current_best_distance

/tmp/ipykernel_803/1496990680.py:11: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = np.array([i.split(" ")[1:] for i in data.split("\n")[6:-2]]).astype(np.float)


In [2]:
mmas = MMAS(city_position=data, initial_pheromone=1000, max_pheromone=100, min_pheromone=50, run_times=500, ant_count=50, alpha=1, beta=2, Q=10000, w=0.1, tau=0.5, q_0=0.8)

/tmp/ipykernel_803/1496990680.py:75: RuntimeWarning: divide by zero encountered in true_divide
  return result, pd.DataFrame(1 / result).replace(float("inf"), 0).values


In [3]:
mmas.run()
mmas.best_distance_lst

运行时间63.188173秒


[28152.568285899408,
 28152.568285899408,
 28152.568285899408,
 28152.568285899408,
 28152.568285899408,
 28152.568285899408,
 28152.568285899408,
 27033.88704314358,
 26798.555461743523,
 26798.555461743523,
 26798.555461743523,
 26013.049034950636,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04823433869,
 25695.04

### CVRP model_routing_1 方法/包报错

In [2]:
from __future__ import print_function
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

###########################
# Problem Data Definition #
###########################
def create_data_model():
  """Stores the data for the problem"""
  data = {}
  # Locations in block units
  _locations = \
          [(4, 4), # depot
           (2, 0), (8, 0), # locations to visit
           (0, 1), (1, 1),
           (5, 2), (7, 2),
           (3, 3), (6, 3),
           (5, 5), (8, 5),
           (1, 6), (2, 6),
           (3, 7), (6, 7),
           (0, 8), (7, 8)]

  demands = [0, # depot
             1, 1, # row 0
             2, 4,
             2, 4,
             8, 8,
             1, 2,
             1, 2,
             4, 4,
             8, 8]

  capacities = [15, 15, 15, 15]

  # Multiply coordinates in block units by the dimensions of an average city block, 114m x 80m,
  # to get location coordinates.
  data["locations"] = [(l[0] * 114, l[1] * 80) for l in _locations]
  data["num_locations"] = len(data["locations"])
  data["num_vehicles"] = 4
  data["depot"] = 0
  data["demands"] = demands
  data["vehicle_capacities"] = capacities
  return data
#######################
# Problem Constraints #
#######################
def manhattan_distance(position_1, position_2):
  """Computes the Manhattan distance between two points"""
  return (
      abs(position_1[0] - position_2[0]) + abs(position_1[1] - position_2[1]))
def create_distance_callback(data):
  """Creates callback to return distance between points."""
  _distances = {}

  for from_node in range(data["num_locations"]):
    _distances[from_node] = {}
    for to_node in range(data["num_locations"]):
      if from_node == to_node:
        _distances[from_node][to_node] = 0
      else:
        _distances[from_node][to_node] = (
            manhattan_distance(data["locations"][from_node],
                               data["locations"][to_node]))

  def distance_callback(from_node, to_node):
    """Returns the manhattan distance between the two nodes"""
    return _distances[from_node][to_node]

  return distance_callback

def create_demand_callback(data):
    """Creates callback to get demands at each location."""
    def demand_callback(from_node, to_node):
        return data["demands"][from_node]
    return demand_callback

def add_capacity_constraints(routing, data, demand_callback):
    """Adds capacity constraint"""
    capacity = "Capacity"
    routing.AddDimensionWithVehicleCapacity(
        demand_callback,
        0, # null capacity slack
        data["vehicle_capacities"], # vehicle maximum capacities
        True, # start cumul to zero
        capacity)
###########
# Printer #
###########
def print_solution(data, routing, assignment):
    """Print routes on console."""
    total_dist = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {0}:\n'.format(vehicle_id)
        route_dist = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = routing.IndexToNode(index)
            next_node_index = routing.IndexToNode(assignment.Value(routing.NextVar(index)))
            route_dist += manhattan_distance(
                data["locations"][node_index],
                data["locations"][next_node_index])
            route_load += data["demands"][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            index = assignment.Value(routing.NextVar(index))

        node_index = routing.IndexToNode(index)
        total_dist += route_dist
        plan_output += ' {0} Load({1})\n'.format(node_index, route_load)
        plan_output += 'Distance of the route: {0}m\n'.format(route_dist)
        plan_output += 'Load of the route: {0}\n'.format(route_load)
        print(plan_output)
    print('Total Distance of all routes: {0}m'.format(total_dist))
########
# Main #
########
def main():
  """Entry point of the program"""
  # Instantiate the data problem.
  data = create_data_model()
  # Create Routing Model
  routing = pywrapcp.RoutingModel(
      data["num_locations"],
      data["num_vehicles"],
      data["depot"])
  # Define weight of each edge
  distance_callback = create_distance_callback(data)
  routing.SetArcCostEvaluatorOfAllVehicles(distance_callback)
# Add Capacity constraint
  demand_callback = create_demand_callback(data)
  add_capacity_constraints(routing, data, demand_callback)
  # Setting first solution heuristic (cheapest addition).
  search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()
  search_parameters.first_solution_strategy = (
      routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
  # Solve the problem.
  assignment = routing.SolveWithParameters(search_parameters)
  if assignment:
    print_solution(data, routing, assignment)
if __name__ == '__main__':
  main()


TypeError: Wrong number or type of arguments for overloaded function 'new_RoutingModel'.
  Possible C/C++ prototypes are:
    operations_research::RoutingModel::RoutingModel(operations_research::RoutingIndexManager const &)
    operations_research::RoutingModel::RoutingModel(operations_research::RoutingIndexManager const &,operations_research::RoutingModelParameters const &)


### CVRP model_routing_2 方法
#### 运行没有问题，但是距离矩阵即调用数据是大问题（600个点且含已定路径）<p>若要用此model可以设定没有连接的路径距离为inf，遍历excel数据计算得矩阵

In [3]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
 
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [
        [0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354,468, 776, 662],
        [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674,1016, 868, 1210],
        [ 776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164,1130, 788, 1552, 754],
        [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822,1164, 560, 1358],
        [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708,1050, 674, 1244],
        [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628,514, 1050, 708],
        [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856,514, 1278, 480],
        [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320,662, 742, 856],
        [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662,320, 1084, 514],
        [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388,274, 810, 468],
        [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764,730, 388, 1152, 354],
        [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114,308, 650, 274, 844],
        [388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194,536, 388, 730],
        [354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0,342, 422, 536],
        [468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536,342, 0, 764, 194],
        [776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274,388, 422, 764, 0, 798],
        [662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730,536, 194, 798, 0],
    ]#距离矩阵 这些地点其实是位于地图上的地点，为了简单化我们将这些地点放到了一个矩形表格中这样处理更加直观
    #一些便于理解， 然后我们会通过一些第三方的地图接口(百度、高德)去计算每个需要访问的地点它们两两之间的
    # 实际距离，并得到一个距离矩阵(distance_matrix)。
    #计算详见：https://developers.google.com/optimization/routing/vrp
    data['num_vehicles'] = 4 #车辆数量
    data['depot'] = 0 #出发点索引
    data['demands'] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8] #每个地点的需求量
    data['vehicle_capacities'] = [15, 15, 15, 15]#每辆车的容量
    return data
 
data = create_data_model()

In [7]:
 
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),data['num_vehicles'], data['depot']) 
routing = pywrapcp.RoutingModel(manager)

def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
 
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

def demand_callback(from_index):
    from_node = manager.IndexToNode(from_index)
    return data['demands'][from_node]
    
 
demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index, # callback_index
    0,  # slack_max
    data['vehicle_capacities'],  # capacity
    True,  # fix_start_cumulative_to_zero
    'Capacity') #维度名称

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
 #注意: 使用 GUIDED_LOCAL_SEARCH 或其他元启发式算法时，需要为求解器设置搜索时间限制——否则求解器不会终止。 可以将时间限制设置为 30 秒。
search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.time_limit.FromSeconds(30)

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))

In [8]:
solution = routing.SolveWithParameters(search_parameters)
 
if solution:
    print_solution(data, manager, routing, solution)

Objective: 6208
Route for vehicle 0:
 0 Load(0) ->  4 Load(4) ->  3 Load(6) ->  1 Load(7) ->  7 Load(15) ->  0 Load(15)
Distance of the route: 1552m
Load of the route: 15

Route for vehicle 1:
 0 Load(0) ->  14 Load(4) ->  16 Load(12) ->  10 Load(14) ->  9 Load(15) ->  0 Load(15)
Distance of the route: 1552m
Load of the route: 15

Route for vehicle 2:
 0 Load(0) ->  12 Load(2) ->  11 Load(3) ->  15 Load(11) ->  13 Load(15) ->  0 Load(15)
Distance of the route: 1552m
Load of the route: 15

Route for vehicle 3:
 0 Load(0) ->  8 Load(8) ->  2 Load(9) ->  6 Load(13) ->  5 Load(15) ->  0 Load(15)
Distance of the route: 1552m
Load of the route: 15

Total distance of all routes: 6208m
Total load of all routes: 60


### CVRP model_docplex实现<p>文件的读取有点问题;最大问题也是路径同上

In [10]:
from math import sqrt
from docplex.mp.model import Model
import matplotlib.pyplot as plt
import numpy as np

In [21]:
#生成class A_computer_science，指定读写文件目录
class A_computer_science:
    def __init__(self):
        self.mDir = "home\\jesse_chen\\anaconda3\\A_computer_science\\"
        self.mDir_input = "i\\"
#定义读数据函数read_data
def read_data(self):#self
    ls_fn = self.mDir + self.mDir_input
    filename = "P-n16-k8.vrp"
    with open(ls_fn + filename, "r") as f:
    #with open(filename, "r") as f:
        data_list = []
        for i in range(9):
            data_list.append(f.readline().strip().split(":"))
        K = int(data_list[2][1])
        N = int(data_list[5][1])
        Q = int(data_list[7][1])
        node_x = []
        node_y = []
        D = []
        C = np.zeros([N, N])
        for i in range(N):
            n, x, y = f.readline().strip().split(" ")
            node_x.append(float(x))
            node_y.append(float(y))
        f.readline()
        for i in range(N):
            n, d = f.readline().strip().split(" ")
            D.append(int(d))
        for i in range(N):
            for j in range(N):
                C[i][j] = sqrt((node_x[i]-node_x[j])**2+(node_y[i]-node_y[j])**2)
        C = np.round(C).astype("int32")
    return N, K, D, C, Q, node_x, node_y

#定义求解数学规划模型的函数solvebycolex
def solve_by_cplex(self, N, K, D, C, Q):
    # Model
    mdl = Model("A_computer_science")
    # Decision variables
    x = mdl.binary_var_cube(N, N, K, name="x")
    y = mdl.binary_var_matrix(N, K, name="y")
    u = mdl.integer_var_matrix(N, K, name="u")
    # Objective
    mdl.minimize(mdl.sum(C[i, j] * x[i, j, k] for i in range(N) for j in range(N) for k in range(K))) # (1)
    # Constraints
    mdl.add_constraints(mdl.sum(y[i, k] for k in range(K)) == 1 for i in range(1, N)) # (2)
    mdl.add_constraint(mdl.sum(y[0, k] for k in range(K)) == K) # (3)
    mdl.add_constraints(mdl.sum(x[i, j, k] for j in range(N)) == mdl.sum(x[j, i, k] for j in range(N))
                        for i in range(N) for k in range(K)) # (4)
    mdl.add_constraints(mdl.sum(x[i, j, k] for j in range(N)) == y[i, k] for i in range(N) for k in range(K)) # (5)
    mdl.add_constraints(mdl.sum(D[i] * y[i, k] for i in range(N)) <= Q for k in range(K)) # (6)
    mdl.add_constraints(u[i, k] - u[j, k] + Q * x[i, j, k] <= Q - D[j] for i in range(1, N) for j in range(1, N)
                        for k in range(K)) # (7)
    mdl.add_constraints(D[i] <= u[i, k] for i in range(1, N) for k in range(K)) # (8)
    mdl.add_constraints(u[i, k] <= Q for i in range(1, N) for k in range(K)) # (8)
    # 约束(9)可加可不加，加了求解速度更快
    mdl.add_constraints(x[i, i, k] == 0 for i in range(N) for k in range(K)) # (9)
    # solve
    mdl.solve(log_output=True)
    # output
    print("obj:%s" % mdl.objective_value)
    tour = {}
    for k in range(K):
        list_i = []
        list_j = []
        list_k = [0]
        for i in range(N):
            for j in range(N):
                if x[i, j, k].solution_value > 0:
                    list_i.append(i)
                    list_j.append(j)
        for l in range(len(list_i)):
            ds = list_i.index(list_k[-1])
            list_k.append(list_j[ds])
        tour[k] = list_k
        print("K=", k, ":", list_k)
    return tour

#定义函数plot绘制路径图
def plot(self, tour, node_x, node_y):
    for k in range(len(tour)):
        list_k = tour[k]
        c = (np.random.rand(), np.random.rand(), np.random.rand())
        for i in range(0, len(list_k) - 1):
            plt.plot([node_x[list_k[i]], node_x[list_k[i + 1]]],[node_y[list_k[i]], node_y[list_k[i + 1]]], color=c, linewidth=2)
    plt.scatter(node_x, node_y)
    plt.show()

#编写主函数
if __name__ == "__main__":
    a = A_computer_science()
    N, K, D, C, Q, node_x, node_y = a.read_data()
    tour = solve_by_cplex(N, K, D, C, Q)
    plot(tour, node_x, node_y)

AttributeError: 'A_computer_science' object has no attribute 'read_data'

### CVRP model_gurovipy实现<p>问题：gurobipy包的license获取<p>优点:node&link的.xlsx文件格式基本一致，路径限制能解决：1、同上直接设置为inf；2、直接调用.xlsx

In [9]:
import pandas as pd
from gurobipy import Model,GRB,tuplelist,tupledict,quicksum

def readXlsxFile(filename):
    df=pd.read_excel(filename,sheet_name=None)
    "读取节点信息"
    df_node=df['node']
    depot=df_node['id'][0]
    N = [depot]
    C = []
    Q = {}
    for i in range(1,df_node.shape[0]):
        id=df_node['id'][i]
        demand=df_node['demand'][i]
        N.append(id)
        C.append(id)
        Q[id]=int(demand)
    N = tuplelist(N)
    C = tuplelist(C)
    Q=tupledict(Q)

    "读取网络弧信息"
    Cost={}
    df_link=df['link']
    for i in range(df_link.shape[0]):
        from_node_id=df_link['from_node_id'][i]
        to_node_id=df_link['to_node_id'][i]
        cost=df_link['link_cost'][i]
        Cost[from_node_id,to_node_id]=cost
    Cost=tupledict(Cost)
    return depot,C,N,Q,Cost

def solveCVRPModel(depot,C,N,Q,Cost,n_vehicles=20,CAP=100):
    """
    :param depot: 车场索引
    :param C: 需求点集合
    :param N: 所有点集合
    :param Q: 需求集合
    :param Cost: 弧运输成本集合
    :param n_vehicles: 最大车辆数量
    :param CAP: 车辆容量
    :return:
    """
    "构建车队"
    K=tuplelist([f'v'+str(k) for k in range(n_vehicles)])
    "实例化模型"
    model=Model('cvrp')
    "添加变量"
    X=model.addVars( N,N,K,vtype=GRB.BINARY,name='X[i,j,k]')
    Y=model.addVars( K,N,vtype=GRB.BINARY,name='Y[k,i]')
    U=model.addVars( K,N,vtype=GRB.CONTINUOUS,name='U[k,i]')
    "目标函数：最小化路径成本"
    z1=quicksum( Cost[i,j]*X[i,j,k] for i in N for j in N for k in K if i!=j)
    model.setObjective(z1,GRB.MINIMIZE)
    "约束：需求覆盖约束"
    model.addConstrs( quicksum( Y[k,i] for k in K ) ==1 for i in C )
    "约束：车辆数量约束"
    model.addConstr( quicksum( Y[k,depot] for k in K) == n_vehicles )
    "约束：流平衡"
    model.addConstrs( quicksum( X[i,j,k] for j in N ) == quicksum( X[j,i,k] for j in N ) for i in N for k in K )
    "约束：决策变量关联"
    model.addConstrs( quicksum( X[i,j,k] for j in N ) == Y[k,i] for i in N for k in K )
    "约束：容量限制"
    model.addConstrs( quicksum( Q[i]*Y[k,i] for i in C ) <= CAP for k in K )
    "约束：破除子环"
    model.addConstrs( U[k,i]-U[k,j]+CAP*X[i,j,k] <= CAP-Q[i] for i in C for j in C for k in K )
    model.addConstrs( Q[i] <= U[k,i] for k in K for i in C )
    model.addConstrs( U[k,i] <= CAP for k in K for i in C)
    "求解"
    model.Params.TimeLimit = 30 # 设置求解时间上限
    model.optimize()
    if model.status == GRB.Status.OPTIMAL or model.status == GRB.Status.TIME_LIMIT:
        for k in K:
            for i in N:
                for j in N:
                    if i!=j :
                        if X[i,j,k].x>0:
                            print("X[{},{},{}]=1".format(i,j,k))
        print("obj:{}".format(model.objVal))
    else:
        print("no solution")


if __name__=='__main__':
    depot,C,N,Q,Cost=readXlsxFile(filename='cvrp_for_gurobi.xlsx')
    solveCVRPModel(depot=depot, C=C, N=N, Q=Q, Cost=Cost, n_vehicles=20, CAP=100)


Set parameter TimeLimit to value 30
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i5-10200H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads



GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license